In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import copy
import os
import datetime
import itertools
import time
%matplotlib inline
mpl.rcdefaults()


In [2]:
df = pd.read_csv("D:/Code/datasets/house prices/train.csv", header=0, encoding='unicode_escape')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [4]:
# Function for initial parsing of df information

# Goals:
# For each col, find out the number of unique values, data type, number of NaN/null values
# create a dictionary output that contains all this information
df_data = {}

for column in list(df): # col name
    data_to_append = {}

    # Unique Values
    number_of_unique_values = df[f"{column}"].nunique()
    data_to_append["Unique Values"] = number_of_unique_values

    # Data Type
    data_type = df[f"{column}"].dtype
    data_to_append["Data Type"] = data_type
    #data type for some columns 


    # Count of Non Null rows per column
    filled_row_count = df[f"{column}"].notnull().sum()
    data_to_append["No. of Filled Rows"] = filled_row_count

    # Min Value - change this to isdigit/isalphnumeric etc to account for other numerical data types
    if data_type == "int64":
        min_value = df[f"{column}"].min()
        data_to_append["Min Value"] = min_value

    # Max Value
    if data_type == "int64":
        max_value = df[f"{column}"].max()
        data_to_append["Max Value"] = max_value

    df_data[f"{column}"] = data_to_append

print(df_data)
print(len(df_data))

output = {
    "numerical_cols": {
        "a": {
            "range": []
        }
    },
    "categorical_cols": {
        "b": {
            "number of unique": 3,

        }
    },
    "datetime_cols": {}
}

{'Id': {'Unique Values': 1460, 'Data Type': dtype('int64'), 'No. of Filled Rows': 1460, 'Min Value': 1, 'Max Value': 1460}, 'MSSubClass': {'Unique Values': 15, 'Data Type': dtype('int64'), 'No. of Filled Rows': 1460, 'Min Value': 20, 'Max Value': 190}, 'MSZoning': {'Unique Values': 5, 'Data Type': dtype('O'), 'No. of Filled Rows': 1460}, 'LotFrontage': {'Unique Values': 110, 'Data Type': dtype('float64'), 'No. of Filled Rows': 1201}, 'LotArea': {'Unique Values': 1073, 'Data Type': dtype('int64'), 'No. of Filled Rows': 1460, 'Min Value': 1300, 'Max Value': 215245}, 'Street': {'Unique Values': 2, 'Data Type': dtype('O'), 'No. of Filled Rows': 1460}, 'Alley': {'Unique Values': 2, 'Data Type': dtype('O'), 'No. of Filled Rows': 91}, 'LotShape': {'Unique Values': 4, 'Data Type': dtype('O'), 'No. of Filled Rows': 1460}, 'LandContour': {'Unique Values': 4, 'Data Type': dtype('O'), 'No. of Filled Rows': 1460}, 'Utilities': {'Unique Values': 2, 'Data Type': dtype('O'), 'No. of Filled Rows': 1460

In [5]:
def df_info(input_df):
    df_data = {}
    for column in list(input_df):
        data_to_append = {}
        # first sort through what type it is 
        ## Numerical data type workflow
        ## what if it's int32 or other variations? should i jsut convert this into a string then run .str.isnumeric operation on it?
        # using isinstance(x,int) would not work in pandas because pandas doesnt treat an int in the same way as in native python

        # Method 1 => hard code and account for all the possible dtypes 
        # Method 2 => convert dtypes to only int64 and float64, and manipulate from there

        if input_df[f"{column}"].dtype == "int64"  or input_df[f"{column}"].dtype == "int32" or input_df[f"{column}"].dtype == "int16" or input_df[f"{column}"].dtype == "int8" or input_df[f"{column}"].dtype == "uint64" or input_df[f"{column}"].dtype == "uint32" or input_df[f"{column}"].dtype == "uint16" or input_df[f"{column}"].dtype == "uint8" or input_df[f"{column}"].dtype == "int_" or input_df[f"{column}"].dtype == "float64" or input_df[f"{column}"].dtype == "float32" or input_df[f"{column}"].dtype == "float16" or input_df[f"{column}"].dtype == "float_":

            data_to_append["Data Type"] = "Numerical"
            
            # range
            max_value = input_df[f"{column}"].max()
            min_value = input_df[f"{column}"].min()
            data_to_append["Range"] = str(min_value) + "-" + str(max_value)

        elif input_df[f"{column}"].dtype == "str" or input_df[f"{column}"].dtype == "O":
            data_to_append["Data Type"] = "Categorical"
            number_of_unique_values = input_df[f"{column}"].nunique()
            data_to_append["Unique Values"] = number_of_unique_values

        #datetime type
        elif input_df[f"{column}"].dtype == "datetime64":
            data_to_append["Data Type"] = "Date Time"

        elif input_df[f"{column}"].dtype == "bool":
            data_to_append["Data Type"] = "Boolean"

    # from there => add more details for each column respectively
        df_data[f"{column}"] = data_to_append
    
    return df_data

print(df_info(df))

output1 = {
    "col a": {
        "data type": "numeric/categorical/datetime" 
        "if numeric - give range"
        "if categorical - give number of unique values (i.e categories) " 

    }

}

{'Id': {'Data Type': 'Numerical', 'Range': '1-1460'}, 'MSSubClass': {'Data Type': 'Numerical', 'Range': '20-190'}, 'MSZoning': {'Data Type': 'Categorical', 'Unique Values': 5}, 'LotFrontage': {'Data Type': 'Numerical', 'Range': '21.0-313.0'}, 'LotArea': {'Data Type': 'Numerical', 'Range': '1300-215245'}, 'Street': {'Data Type': 'Categorical', 'Unique Values': 2}, 'Alley': {'Data Type': 'Categorical', 'Unique Values': 2}, 'LotShape': {'Data Type': 'Categorical', 'Unique Values': 4}, 'LandContour': {'Data Type': 'Categorical', 'Unique Values': 4}, 'Utilities': {'Data Type': 'Categorical', 'Unique Values': 2}, 'LotConfig': {'Data Type': 'Categorical', 'Unique Values': 5}, 'LandSlope': {'Data Type': 'Categorical', 'Unique Values': 3}, 'Neighborhood': {'Data Type': 'Categorical', 'Unique Values': 25}, 'Condition1': {'Data Type': 'Categorical', 'Unique Values': 9}, 'Condition2': {'Data Type': 'Categorical', 'Unique Values': 8}, 'BldgType': {'Data Type': 'Categorical', 'Unique Values': 5}, 'H

In [9]:
## Fully automated function (WIP)
start = time.time()
combined_col = []
pri_cols = ["Division", "Location", "ResourceLevel"]
sec_col = ["Division", "Location", "Support from Company"]
list_of_val_cols_calc = ["Department", "Status"]
list_of_val_cols_nocalc = ["Service Years", "Age"]

current_dir = os.getcwd()
current_datetime = datetime.datetime.now().strftime("%d.%m.%Y_%H%M")
dest_path = current_dir+'\\'+current_datetime + "_Visualisation"
if not os.path.exists(dest_path):
    os.makedirs(dest_path)

mapping = {
    "MSSubClass": "categorical",
    "MSZoning": "categorical",
    "Street": "categorical",
    "LotShape": "categorical",
    "LandContour": "categorical",
    "OverallQual": "numerical",
    "OverallCond": "numerical",
    "YearBuilt": "datetime",
    "YrSold": "datetime"
}

### START OF SAVING VISUALS FUNCTION
# --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
def save_viz(file_name):
    locs, labels = plt.xticks()
    plt.setp(labels, rotation=90, horizontalalignment = "right")
    plt.savefig(os.path.join(dest_path, file_name), dpi=300, bbox_inches='tight')
# --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
### END OF SAVING VISUALS FUNCTION

### START OF PLOTTING FUNCTION
# --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
def label_checker(x_label: str, hue_label: str):
    if x_label != hue_label:
        return True

def str_replacing(input_str: str):
    str1 = input_str.replace(" ", "_").replace("/", "_")
    return str1

def hist_plot(viz_df: pd.DataFrame, x_label: str, hue_label: str, y_label: str):
    hist_fig = sns.displot(data=viz_df, x=x_label, y=y_label, kind="hist", hue=hue_label, palette="bright")
    x_label = str_replacing(x_label)
    y_label = str_replacing(y_label)
    file_name = f"HistPlot_of_{x_label}_and_{y_label}.png"
    save_viz(file_name)
    plt.close()

def kde_plot(viz_df: pd.DataFrame, x_label: str, hue_label: str, y_label: str):
    kde_fig = sns.displot(data=viz_df, x=x_label, y=y_label, kind="kde", hue=hue_label, palette="bright") 
    x_label = str_replacing(x_label)
    y_label = str_replacing(y_label)
    file_name = f"KDEPlot_of_{x_label}_and_{y_label}.png"
    save_viz(file_name)
    plt.close()

def bar_plot(viz_df: pd.DataFrame, x_label: str, hue_label: str, y_label: str):  
    bar_fig = sns.catplot(data=viz_df, x=x_label, y=y_label, kind="bar", hue=hue_label, palette="bright")
    #print("x label is: " + str(x_label) + "---- y label is: " + str(y_label) + "\n")
    x_label = str_replacing(x_label)
    y_label = str_replacing(y_label)
    file_name = f"BarPlot_of_{x_label}_and_{y_label}.png"
    save_viz(file_name)
    plt.close()

def box_plot(viz_df: pd.DataFrame, x_label: str, hue_label: str, y_label: str):
    box_fig = sns.boxplot(data=viz_df, x=x_label, y=y_label, hue=hue_label, palette="bright")
    x_label = str_replacing(x_label)
    y_label = str_replacing(y_label)
    file_name = f"BoxPlot_of_{x_label}_and_{y_label}.png"
    save_viz(file_name)
    plt.close()

def swarm_plot(viz_df: pd.DataFrame, x_label: str, hue_label: str, y_label: str):
    swarm_fig = sns.catplot(data=viz_df, x=x_label, y=y_label, kind="swarm", hue=hue_label, palette="bright")
    file_name = f"SwarmPlot_of_{x_label}_and_{y_label}.png"
    save_viz(file_name)
    plt.close()

def scatter_plot(viz_df: pd.DataFrame, x_label: str, hue_label: str, y_label: str):
    scatter_fig = sns.relplot(data=viz_df, x=x_label, y=y_label, kind="scatter", hue=hue_label, palette="bright")
    x_label = str_replacing(x_label)
    y_label = str_replacing(y_label)
    file_name = f"ScatterPlot_of_{x_label}_and_{y_label}.png"
    save_viz(file_name)
    plt.close()

def line_plot(viz_df: pd.DataFrame, x_label: str, hue_label: str, y_label: str):
    line_fig = sns.relplot(data=viz_df, x=x_label, y=y_label, kind="line", hue=hue_label, palette="bright")
    x_label = str_replacing(x_label)
    y_label = str_replacing(y_label)
    file_name = f"LinePlot_of_{x_label}_and_{y_label}.png"
    save_viz(file_name)
    plt.close()

def lm_plot(viz_df: pd.DataFrame, x_label: str, hue_label: str, y_label: str):
    lm_fig = sns.lmplot(data=viz_df, x=x_label, y=y_label, hue=hue_label, palette="bright")
    x_label = str_replacing(x_label)
    y_label = str_replacing(y_label)
    file_name = f"LMPlot_of_{x_label}_and_{y_label}.png"
    save_viz(file_name)
    plt.close()

def ecdf_plot(viz_df: pd.DataFrame, x_label: str, hue_label: str):
    ecdf_fig = sns.displot(data=viz_df, x=x_label, kind="ecdf", hue=hue_label, palette="bright")
    x_label = str_replacing(x_label)
    file_name = f"ECDFPlot_of_{x_label}.png"
    save_viz(file_name)
    plt.close()

def viz(viz_df: pd.DataFrame, x_label_list: list, hue_label: list, y_label_list: list):
    hue_label = ''.join(hue_label)
    start = time.time()
    for (x_label, y_label) in itertools.product(x_label_list, y_label_list):

        if (mapping[x_label] == "categorical" or mapping[x_label] == "datetime") and (mapping[y_label] == "numerical") and (label_checker(x_label, hue_label)):
            bar_plot(viz_df, x_label, hue_label, y_label)
            box_plot(viz_df, x_label, hue_label, y_label)

        elif (mapping[x_label] == "numerical" or mapping[x_label] == "datetime") and (mapping[y_label] == "numerical") and (label_checker(x_label, hue_label)):
            kde_plot(viz_df, x_label, hue_label, y_label)
            line_plot(viz_df, x_label, hue_label, y_label)
        
        elif (mapping[x_label] == "numerical" or mapping[x_label] == "datetime") and (mapping[y_label] == "numerical" or mapping[y_label] == "datetime") and (label_checker(x_label, hue_label)):
            lm_plot(viz_df, x_label, hue_label, y_label)

        elif (mapping[x_label] == "categorical" or mapping[x_label] == "datetime") and (mapping[x_label] == "numerical") and (label_checker(x_label, hue_label)):
            hist_plot(viz_df, x_label, hue_label, y_label)
            scatter_plot(viz_df, x_label, hue_label, y_label)
            swarm_plot(viz_df, x_label, hue_label, y_label)
            
        elif (mapping[x_label] == "categorical" or mapping[x_label] == "numerical" or mapping[x_label] == "datetime") and (label_checker(x_label, hue_label)):
            ecdf_plot(viz_df, x_label, hue_label)
            
    end = time.time()
    #print(end-start)
# --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
### END OF PLOTTING FUNCTION

### START OF CALCULATED TABLE FUNCTION
# --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
def table_calc(input_df_calc: pd.DataFrame, pri_cols_calc: list,sec_col_calc: list, val_col_calc: list, val_col_nocalc: list = []):
    
    original_val_col_calc = copy.deepcopy(val_col_calc)
    print(pri_cols_calc)
    print(sec_col_calc)
    print("\n" + str(val_col_calc))
    try:
        group_by_col = pri_cols_calc + sec_col_calc
        group_by_col = list(set(group_by_col))
        combined_col = group_by_col + val_col_calc + val_col_nocalc
    except Exception as e:
        print("Error was: " + str(e))
    new_df = pd.DataFrame()
    test_df = input_df_calc.loc[:, combined_col]
    test_df = test_df.T.drop_duplicates().T
    # for loop for val cols
    val_col_calc = copy.deepcopy(original_val_col_calc)
    for val in val_col_calc: 
        # for loop to generate cols for val cols

        unique_vals = []
        new_col_name = ""
        new_col_name_list = []
        unique_vals = list(set(test_df[val])) # gives me the names of the unique vals within the values col
        num_unique_vals = len(unique_vals)
        for i in range(num_unique_vals):
            eval_col = np.where(test_df[val] == unique_vals[i], 1, 0)# unique vals is status, voluntary words
            new_col_name = f"Count_of_{val}_{unique_vals[i]}"
            eval_col = eval_col.flatten()
            new_df[new_col_name] = eval_col
            new_col_name_list.append(new_col_name)
    
    test_df = pd.concat([test_df, new_df], axis=1, join="inner")
    #test_df = test_df.groupby(group_by_col, as_index=True).transform("sum")
    for i in new_col_name_list:
        val_col_calc.append(i)
        mapping[i] = "numerical"
            
    test_df = test_df.drop_duplicates(group_by_col)
    val_col_calc += val_col_nocalc
    viz(test_df, pri_cols_calc, sec_col_calc, val_col_calc)
# --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
### END OF CALCULATED TABLE FUNCTION

### START OF INITIALISATION FUNCTION
# --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
def initialisation(calc_needed: bool, input_df_initialisation: pd.DataFrame, df_mapping: dict, pri_cols_initialisation: list = [], sec_col_initialisation: list = [], val_col_initialisation: list = [], val_col_initialisation_nocalc: list = []):
    input_df_initialisation = input_df_initialisation.replace("/", "_", regex=True).replace(" ", "_", regex=True)
    counter = 0
    
    if (pri_cols_initialisation == []) and (sec_col_initialisation == []) and (val_col_initialisation == [] and (val_col_initialisation_nocalc == [])):
        for keys,values in df_mapping.items():
            try:
                if values == "categorical":
                    pri_cols_initialisation.append(keys)
                    val_col_initialisation.append(keys)
                    sec_col_initialisation.append(keys)

                elif values == "numerical" or values == "datetime":
                    val_col_initialisation_nocalc.append(keys)
        
            except Exception as e:
                print("Your error is: " + str(e))
                break
        # sec_col_initialisation = pri_cols_initialisation[0]
    initial_val_cols = copy.deepcopy(val_col_initialisation)

    if counter != 0:
        val_col_initialisation = copy.deepcopy(initial_val_cols)

        ## ITERTOOLS WIP
    # for a, b in itertools.product(pri_cols_initialisation, sec_col_initialisation):
    #     if counter != 0:
    #         val_col_initialisation = copy.deepcopy(initial_val_cols)
    #     # if calc_needed == True:
    #     #     table_calc(input_df_initialisation, )
    #     print("a is:" + str(a) + "---------- b is:" + str(b) + "\n")


    if len(sec_col_initialisation) != 1:
        lists_of_sec_cols = [sec_col_initialisation[x: x+1] for x in range(0, len(sec_col_initialisation), 1)]
        for a in range(len(lists_of_sec_cols)):
            sec_col_initialisation = lists_of_sec_cols[a]
            if counter != 0:
                val_col_initialisation = copy.deepcopy(initial_val_cols)
            if calc_needed == True:
                table_calc(input_df_initialisation, pri_cols_initialisation, sec_col_initialisation, val_col_initialisation, val_col_initialisation_nocalc)
                counter += 1
            else:
                val_col_initialisation = val_col_initialisation + val_col_initialisation_nocalc
                viz(input_df_initialisation, pri_cols_initialisation, sec_col_initialisation, val_col_initialisation)
    if calc_needed == True:
        table_calc(input_df_initialisation, pri_cols_initialisation, sec_col_initialisation, val_col_initialisation, val_col_initialisation_nocalc)
    else:
        val_col_initialisation = val_col_initialisation + val_col_initialisation_nocalc
        viz(input_df_initialisation, pri_cols_initialisation, sec_col_initialisation, val_col_initialisation)
# --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
### END OF INITIALISATION FUNCTION
initialisation(1, df, mapping)
#nitialisation(1, df, mapping, pri_cols, sec_col, list_of_val_cols_calc, list_of_val_cols_nocalc) # only this will not have an int
end = time.time()
print(end-start)


['MSSubClass', 'MSZoning', 'Street', 'LotShape', 'LandContour']
['MSSubClass']

['MSSubClass', 'MSZoning', 'Street', 'LotShape', 'LandContour']
['MSSubClass', 'MSZoning', 'Street', 'LotShape', 'LandContour']
['MSZoning']

['MSSubClass', 'MSZoning', 'Street', 'LotShape', 'LandContour']
['MSSubClass', 'MSZoning', 'Street', 'LotShape', 'LandContour']
['Street']

['MSSubClass', 'MSZoning', 'Street', 'LotShape', 'LandContour']
['MSSubClass', 'MSZoning', 'Street', 'LotShape', 'LandContour']
['LotShape']

['MSSubClass', 'MSZoning', 'Street', 'LotShape', 'LandContour']
['MSSubClass', 'MSZoning', 'Street', 'LotShape', 'LandContour']
['LandContour']

['MSSubClass', 'MSZoning', 'Street', 'LotShape', 'LandContour']
['MSSubClass', 'MSZoning', 'Street', 'LotShape', 'LandContour']
['LandContour']

['MSSubClass', 'MSZoning', 'Street', 'LotShape', 'LandContour']
273.5506868362427
